In [1]:
import cv2
import os
from os import listdir
from os.path import isfile, join
from PIL import Image as Image
import numpy as np

from scipy.optimize import curve_fit

def relit(x, a, b):
    return (a * x.astype(np.float)/255 + b)*255



from matplotlib import pyplot as plt
def plshow(im,title='MINE'):
    if len(im.shape)>2:
  #      plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.imshow(im)
    else:
        plt.imshow(im,cmap='gray')
    plt.title(title)
    plt.rcParams["figure.figsize"] = (80,12)
    plt.show()

sd_path = 'dataset/ISTD/train_A'
mask_path = 'dataset/ISTD/train_B'
sdfree_path = 'dataset/ISTD/train_C_fixed_ours'

sd_path = '/home/balin/exper/shadow_removal/Dataset/ISTD_Dataset/train/train_A'
mask_path = '/home/balin/exper/shadow_removal/Dataset/ISTD_Dataset/train/train_B'
sdfree_path = '/home/balin/exper/shadow_removal/Dataset/ISTD_Dataset/train/train_C'

out = 'dataset/ISTD/train_params/'

if not os.path.exists(out):
    os.makedirs(out)

im_list  =  [f for f in listdir(sd_path) if isfile(join(sd_path, f)) and f.endswith('png')]
print(len(im_list),im_list[0])
kernel = np.ones((5,5),np.uint8)


def im_relit(Rpopt,Gpopt,Bpopt,dump):
    #fc this shit
    sdim = dump.copy()
    sdim.setflags(write=1)
    sdim = sdim.astype(np.float)
    sdim[:,:,0] = (sdim[:,:,0]/255) * Rpopt[0] + Rpopt[1]
    sdim[:,:,1] = (sdim[:,:,1]/255) * Gpopt[0] + Gpopt[1]
    sdim[:,:,2] = (sdim[:,:,2]/255) * Bpopt[0] + Bpopt[1]
    sdim = sdim*255
   # print(np.amin(sdim),np.amax(sdim))
    return sdim

errors= []
for im in im_list:
    sd = np.asarray(Image.open(join(sd_path,im)))
    mean_sdim = np.mean(sd,axis=2)
    
    mask_ori = np.asarray(Image.open(join(mask_path,im)))
    mask = cv2.erode(mask_ori ,kernel,iterations = 2)

    
    sdfree = np.asarray(Image.open(join(sdfree_path,im)))
    mean_sdfreeim = np.mean(sdfree,axis=2)
    
    #pixels for regression funtion
    i, j = np.where(np.logical_and(np.logical_and(np.logical_and(mask>=1,mean_sdim>5),mean_sdfreeim<230),np.abs(mean_sdim-mean_sdfreeim)>10))

    source = sd*0
    source[tuple([i,j])] = sd[tuple([i,j])] 
    target = sd*0
    target[tuple([i,j])]= sdfree[tuple([i,j])]
    
    R_s = source[:,:,0][tuple([i,j])]
    G_s = source[:,:,1][tuple([i,j])]
    B_s = source[:,:,2][tuple([i,j])]
    
    R_t = target[:,:,0][tuple([i,j])]
    G_t = target[:,:,1][tuple([i,j])]
    B_t = target[:,:,2][tuple([i,j])]
    
    c_bounds = [[1,-0.1],[10,0.5]]

    
    Rpopt, pcov = curve_fit(relit, R_s, R_t,bounds=c_bounds)
    Gpopt, pcov = curve_fit(relit, G_s, G_t,bounds=c_bounds)
    Bpopt, pcov = curve_fit(relit, B_s, B_t,bounds=c_bounds)
    
    
    relitim = im_relit(Rpopt,Gpopt,Bpopt,sd)
    
    #final = sd.copy()
    #final[tuple([i,j])] = relitim[tuple([i,j])]
    #final[final>255] =255
    #final[final<0] = 0

    #plshow(final)
    error = np.mean(np.abs(relitim[tuple([i,j])].astype(np.float) - sdfree[tuple([i,j])]).astype(np.float))
    print(error,Rpopt,Gpopt,Bpopt)
    f = open(join(out,im+'.txt'),"a")
    f.write("%f %f %f %f %f %f"%(Rpopt[1],Rpopt[0],Gpopt[1],Gpopt[0],Bpopt[1],Bpopt[0]))
    f.close()
             
  #  print(error)
    errors.append(error)
                    
    
print(np.mean(errors))
#no bound - 8.55
#### y_bound ###error
#    0.5        8.86
#    0.1        15.692271753155671    
#    0.25       10.830443545867785
#    1          8.86

1330 11-11.png
2.7337523439313327 [1.24381949 0.0449121 ] [1.1545337  0.05261563] [1.11263318 0.04797625]
14.309232321595259 [1.19537021 0.27767171] [1.15171877 0.23819064] [1.10994462 0.19815077]
10.085786451395906 [1.         0.36011569] [1.         0.33238398] [1.         0.28986589]
5.02494750980352 [1.19509093 0.17953185] [1.15172574 0.13746585] [1.00995447 0.12998708]
11.746952964526319 [1.12776664 0.23587642] [1.08605944 0.21052226] [1.04228271 0.16253346]
8.9019632656465 [1.         0.35103997] [1.         0.32556975] [1.0011038  0.28496644]
8.38110056535951 [1.         0.15171382] [1.         0.16546316] [1.         0.07502833]
10.110799029367767 [1.63136412 0.08326089] [1.50472682 0.08314616] [1.36260601 0.07359035]
13.849872075794055 [1.15832197 0.19378572] [1.06910662 0.22294764] [1.27584907 0.08599777]
8.518317108887457 [1.47733992 0.30777627] [1.3225539  0.30364514] [1.56765879 0.16428187]
14.605685680168298 [1.        0.3214281] [1.         0.28458508] [1.40560444 0.1079

9.043096698001046 [1.        0.2413976] [1.        0.2172476] [1.         0.17808611]
12.078614953631348 [3.53329442 0.25847085] [2.78572015 0.27321377] [2.26692995 0.24811801]
8.779312891728578 [ 1.63951908e+00 -9.22426647e-04] [1.24063181 0.11164209] [1.         0.27209836]
10.485100248192145 [1.48973592 0.103584  ] [1.41542789 0.09647637] [1.35091209 0.06324824]
5.536146071642291 [1.12944149 0.12510485] [1.10651916 0.0913406 ] [1.11757431 0.01374691]
2.760746388146383 [1.05032902 0.1258081 ] [ 1.20849525 -0.00859103] [ 1.0913209  -0.00410555]
8.005926023951865 [1.29807527 0.14814194] [1.25380711 0.1236408 ] [1.22437475 0.06105544]
5.5453661528641245 [1.35410124 0.18238555] [1.15977932 0.1707985 ] [1.11363791 0.11605332]
8.73466379109853 [1.29167062 0.10360741] [1.12085709 0.13272599] [ 1.30870662 -0.01082194]
6.749894548097503 [1.13278215 0.12556142] [1.14302108 0.10194227] [1.19265    0.03094783]
3.3967744196573055 [1.26952854 0.04838205] [1.24112903 0.05212457] [1.21497971 0.04607

5.660884680244155 [1.26546346 0.06836735] [1.19079858 0.06358907] [1.12107939 0.04884261]
14.471962665891388 [1.1247542  0.20918634] [1.0508054  0.23866692] [1.28588118 0.09178111]
6.806664081586923 [1.27441183 0.22696347] [1.13704069 0.186722  ] [1.07210542 0.12946351]
5.058498345180722 [1.18394061 0.27119978] [1.10773148 0.23546487] [1.03460533 0.20444445]
6.030985965509696 [1.24142071 0.04723979] [1.2155356  0.05164759] [1.18344969 0.05109911]
7.613875121213023 [1.        0.3284662] [1.         0.27775719] [1.         0.20444111]
18.99728573197747 [1.         0.06944848] [1.         0.04008963] [ 1.         -0.04357582]
4.9396918200089965 [1.10191619 0.18392322] [1.08525711 0.15343556] [1.02732962 0.07269988]
9.680984568701724 [1.23604576 0.2686347 ] [1.19763451 0.23465049] [1.14661689 0.17512953]
4.039535483029613 [1.06603463 0.35925213] [1.         0.34713716] [1.       0.311395]
6.808791123784319 [1.33718647 0.08868678] [1.         0.15200884] [1.         0.08342131]
8.5207953864

15.071747596234278 [1.         0.35394318] [1.         0.31306088] [1.         0.25655991]
5.818876570860489 [1.32616456 0.0885734 ] [1.23506761 0.08773284] [1.12668959 0.08045957]
5.652322555405759 [1.28185159 0.08927917] [1.26353391 0.04133712] [1.15288887 0.03309956]
12.156123063058951 [1.         0.30049493] [1.         0.26237321] [1.00842319 0.2065506 ]
20.44794384567486 [1.0604247 0.144727 ] [1.         0.14862424] [1.03711652 0.02218396]
6.733771911010999 [1.1058144  0.06447325] [1.06898894 0.06006472] [1.04324733 0.04307328]
3.5748873032370865 [1.45674124 0.05311642] [1.         0.18374422] [1.         0.07527432]
3.1824457236160866 [1.08584539 0.05908894] [1.02327545 0.05081999] [1.01189471 0.03088833]
6.6128859942961515 [1.         0.24282589] [1.         0.21944455] [1.         0.18090024]
8.309406764371834 [1.32926945 0.2902326 ] [1.31083563 0.23084543] [1.15395161 0.23111397]
8.704811596123431 [1.64365633 0.27302747] [1.50275059 0.24944441] [1.61618151 0.15641441]
11.2551

7.969928821358154 [1.30838005 0.15257581] [1.26526027 0.13341665] [1.18878064 0.13461041]
4.548165104297757 [1.11110458 0.16791715] [1.13523745 0.07609058] [ 1.13924924 -0.02141951]
14.713840606870699 [1.         0.40064917] [1.         0.36092339] [1.        0.3020897]
10.843069770733031 [1.         0.38296798] [1.         0.35458707] [1.         0.30521973]
10.77884945015864 [1.         0.31451337] [1.         0.25590183] [1.         0.20062473]
10.54459455187432 [1.3130231  0.30048085] [1.16326397 0.2900402 ] [1.02972695 0.26281387]
6.011035727837922 [1.03150247 0.2745741 ] [1.05419014 0.24464229] [1.04058116 0.2166872 ]
6.6076458818721635 [1.08369294 0.16135208] [1.05177006 0.15041208] [1.01889165 0.13414803]
7.431357713346608 [2.34079177 0.24538992] [2.35837851 0.19167105] [1.83883916 0.18384869]
9.477055288709611 [1.52722169 0.18269034] [1.47518518 0.14235632] [1.29592037 0.12929615]
6.438511263477937 [1.1277185  0.05086577] [1.08087473 0.05157563] [1.05041479 0.03906726]
8.85149

...

7.87004510635923 [1.         0.21676004] [1.         0.18420298] [1.         0.12531191]
3.7800494147119257 [1.60525284 0.3203771 ] [1.49855898 0.29505867] [1.25644693 0.27283727]
10.08572239242512 [1.        0.2447141] [1.         0.20553471] [1.         0.15198472]
7.107108858240195 [1.20752467 0.19497197] [1.16422757 0.17854429] [1.07418035 0.15468061]
6.319723546110594 [1.13436375 0.10838871] [1.12865353 0.09228593] [1.11622767 0.06705994]
11.198906234615936 [1.         0.34967578] [1.         0.30911513] [1.         0.25891445]
9.528401418515394 [1.         0.24555718] [1.         0.21725462] [1.         0.17716192]
8.181145174840427 [1.         0.33453851] [1.         0.30281275] [1.         0.26585612]
11.316065831870269 [1.        0.3488616] [1.         0.29919341] [1.         0.22238485]
11.683791754276571 [1.45868774 0.23909107] [1.42018402 0.18672665] [1.26413221 0.1454603 ]
4.231638209062736 [1.14965148 0.18281614] [1.1918021  0.12386538] [1.18795314 0.07611   ]
10.35838657

8.914234663832186 [1.46525589 0.00698782] [1.06188855 0.17190099] [1.        0.1891561]
6.976946207797524 [1.10286591 0.07114945] [1.06535581 0.06931986] [1.04630439 0.05029129]
12.159952896027981 [1.07831273 0.23430522] [1.0393249  0.20870004] [1.00199031 0.15894079]
4.959556215101155 [ 2.45539103 -0.02197833] [1.93408566 0.03336161] [1.64867133 0.0485331 ]
7.180810989767195 [1.03755241 0.21796405] [1.04990932 0.16436993] [1.         0.13859447]
3.471688806826051 [1.73892282 0.03260765] [1.77815244e+00 3.05375719e-04] [1.36867773 0.01872795]
14.718766154372261 [1.         0.42635487] [1.14960851 0.33227763] [1.         0.31683155]
5.348889247498803 [1.         0.19843646] [ 1.34700892 -0.01121774] [ 1.18697793 -0.00347305]
16.452012846068936 [1.11026436 0.19612627] [1.02551536 0.2263148 ] [1.26314573 0.06673648]
7.147119229259793 [1.20599688 0.30834521] [1.10363841 0.31007184] [1.0291271 0.2947647]
8.82528927120314 [1.44904081 0.21720023] [1.28570088 0.1937311 ] [1.21669265 0.12742631

10.747606194932722 [1.         0.38579391] [1.         0.33020855] [1.         0.27101005]
9.541975427286701 [1.37590959 0.15428557] [1.3415392  0.14704114] [1.29456344 0.12026006]
5.876817652548014 [1.         0.09555523] [1.         0.07483474] [1.03108228 0.03081488]
7.831967227767757 [1.         0.33478034] [1.26464552 0.20869589] [1.19857121 0.14211351]
8.76852958048748 [1.53156607 0.22483762] [1.40241731 0.19431847] [1.07712975 0.18430051]
10.758253349669904 [1.         0.10130961] [1.01926063 0.0842297 ] [1.01716081 0.07597476]
9.67549057422101 [1.50123364 0.19829355] [1.36674064 0.18631413] [1.32050418 0.13132777]
7.585318160342969 [1.09573316 0.27654991] [1.13300322 0.21427532] [1.1104077  0.15786001]
10.285702550184592 [1.        0.4041233] [1.         0.37428258] [1.         0.33504583]
6.671279699035271 [1.4120207  0.25912619] [1.19322737 0.22188287] [1.02024863 0.18647308]
8.318696118856085 [1.06371548 0.22275413] [1.03592448 0.1922099 ] [1.         0.13464903]
7.907711458

7.0089784446364165 [1.57445281 0.22349309] [1.34956176 0.20783177] [1.13116993 0.18024573]
13.177207837105556 [1.11246306 0.3598682 ] [1.0193778  0.33483332] [1.         0.27148534]
14.628099848426675 [1.         0.43346933] [1.10542612 0.36837029] [1.09617158 0.31420201]
10.012329699477196 [1.07188983 0.21291101] [1.03560492 0.17706938] [1.         0.13121349]
11.468020885731818 [1.10056893 0.32955619] [1.03952975 0.30338292] [1.         0.25486456]
6.387041718901235 [1.16980114 0.14540405] [1.19176328 0.09495291] [1.12058474 0.07930526]
4.87839025433593 [1.15956213 0.09485424] [1.12811977 0.07247613] [1.09440209 0.06439761]
5.739545526856406 [1.27455264 0.15131228] [1.21555228 0.13288202] [1.35507338 0.01190099]
7.406060057114057 [1.18566876 0.15637592] [1.15829453 0.13923469] [1.11637159 0.110767  ]
3.0323149439269295 [1.07046217 0.0603065 ] [1.01323185 0.04990628] [1.         0.02915291]
4.942622722287428 [1.18237261 0.23486982] [1.08717236 0.2348902 ] [1.         0.22689673]
4.706

6.6632873332512945 [1.30574769 0.11366904] [1.27004894 0.07768856] [ 1.27592784 -0.00490569]
8.09566193841586 [1.34711384 0.21500298] [1.13002646 0.20717114] [1.00236566 0.183003  ]
9.32033366854012 [1.2713119  0.16688304] [1.24237559 0.1100341 ] [1.10497329 0.07191708]
9.423621493586614 [1.14991163 0.2737304 ] [1.04233394 0.25539382] [1.         0.21309476]
7.625120901669816 [1.16251274 0.06239505] [1.12362106 0.06177418] [1.08653631 0.0536167 ]
9.315852019994432 [1.34210175 0.2928935 ] [1.30946016 0.22265368] [1.15105891 0.22711072]
12.669195089868431 [1.14027316 0.31359234] [1.48800076 0.15297977] [1.35608957 0.06757077]
6.071335279441856 [1.64997678 0.19133162] [1.27939206 0.19958607] [1.30863889 0.0856668 ]
3.962175652581529 [1.3425594  0.08904949] [1.29504268 0.07908399] [1.17289306 0.0291482 ]
4.566008965844901 [1.46851808 0.24579658] [1.28559796 0.20631841] [1.08105868 0.17380052]
12.456430271556947 [1.25558955 0.27067526] [1.27587426 0.22449003] [1.13651075 0.19388499]
15.6843

In [6]:
import os
cwd = os.getcwd()
print(cwd)

/home/balin/exper/shadow_removal/data_processing
